# Obtaining Data

## Stakeholders 
The Seattle Police Department has reached out to determine the likelihood of Terry Stops preventing a crime by arresting suspects under the basis of "probable cause". In an effort to improve race relations, they also want to see how much of a role racial discrimination plays in who gets stopped.  

## Objectives: 
* Determine whether Terry Stops are preventing crimes
* Determine if there is a relationship between Terry Stops and a subject's race
* Do the differences in races of the police officer and the subject play a role?
* 

Ok, let's check out this data!

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [2]:
df = pd.read_csv('CSV_Files/Terry_Stops.csv')
df.head()

,Subject Age Group,Subject ID,GO / SC Num,Terry Stop ID,Stop Resolution,Weapon Type,Officer ID,Officer YOB,Officer Gender,Officer Race,...,Reported Time,Initial Call Type,Final Call Type,Call Type,Officer Squad,Arrest Flag,Frisk Flag,Precinct,Sector,Beat
0,-,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,-,-,-,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,-,-1,20150000001463,28806,Field Contact,None,5670,1965,M,White,...,07:59:00,-,-,-,NaN,N,N,-,-,-
2,-,-1,20150000001516,29599,Field Contact,None,4844,1961,M,White,...,19:12:00,-,-,-,NaN,N,-,-,-,-
3,-,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,-,-,-,NaN,N,N,-,-,-
4,-,-1,20150000001739,33155,Field Contact,None,6973,1977,M,White,...,00:41:00,-,-,-,NaN,N,N,-,-,-


That's... A lot of missing data...

In [3]:
df.shape

(43512, 23)

We see a lot of place holder values, let's look for any NaNs in our dataset.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43512 entries, 0 to 43511
Data columns (total 23 columns):
Subject Age Group           43512 non-null object
Subject ID                  43512 non-null int64
GO / SC Num                 43512 non-null int64
Terry Stop ID               43512 non-null int64
Stop Resolution             43512 non-null object
Weapon Type                 43512 non-null object
Officer ID                  43512 non-null object
Officer YOB                 43512 non-null int64
Officer Gender              43512 non-null object
Officer Race                43512 non-null object
Subject Perceived Race      43512 non-null object
Subject Perceived Gender    43512 non-null object
Reported Date               43512 non-null object
Reported Time               43512 non-null object
Initial Call Type           43512 non-null object
Final Call Type             43512 non-null object
Call Type                   43512 non-null object
Officer Squad               42968 non-null ob

Officer Squad has 544 NaN values.  To save time we'll make a function that shows us the value counts of each column

In [5]:
def col_values(df):
    """
    For use in Preprocessing and cleaning to find placeholder values
    Input: Data frame
    Output: Counts of unique values for each column
    """
    for col in df.columns:
        print(col)
        print('\n')
        print(df[col].value_counts())
        print('-------------------------------------------------------')
        print('\n')

In [6]:
col_values(df)

Subject Age Group


26 - 35         14420
36 - 45          9153
18 - 25          8875
46 - 55          5599
56 and Above     2185
1 - 17           1908
-                1372
Name: Subject Age Group, dtype: int64
-------------------------------------------------------


Subject ID


-1              34676
 7726859935        18
 7727117712        12
 7753260438        11
 7726318196         8
                ...  
 7730602336         1
 7758922092         1
 7727677812         1
 9640739188         1
 12095084261        1
Name: Subject ID, Length: 7045, dtype: int64
-------------------------------------------------------


GO / SC Num


20150000190790    16
20160000378750    16
20180000134604    14
20170000132836    13
20190000441736    13
                  ..
20190000076830     1
20150000379928     1
20180000002079     1
20180000333849     1
20180000071981     1
Name: GO / SC Num, Length: 33910, dtype: int64
-------------------------------------------------------


Terry Stop ID


130800

### Notes:
**Age Groups**: 1372 people with unknown age

**Subject ID**: 34,676 people that are not assigned an ID number, also some people stopped multiple times

**General Offense**: Repeated values, need to look into why

**Terry Stop ID**: Repeated values. These duplicates should definitely be removed.

**Stop Resolution**: No NaNs or place holders, but distinction needs to be made between physical and non-custodial arrest.

**Weapon Type**: Needs to be cleaned up and binned. Need to create 'unknown' to account for placeholders

**Officer ID**: Clean. This data may be irrelevant.

**Officer YOB**: Clean.

**Officer Gender**: 11 counts of 'N'.  Unable to determine if this means unknown, non-gender binary, etc.  It's a tiny amount of data, just drop it.

**Reported Date**: Clean; but delete the Time counter and extract the year to feature engineer the age of the officer.

**Reported Time**: Clean

**Initial Call/Final/Call Type**: The consistent appearance of 12,828 seems that these were calls made **BEFORE** the integration of the **Computer Assisted Dispatch** system. Although this is a lot of data to just drop, it could cause problems with interpretation since we can't tell if this was a result of an officer being proactive or responding to a call.

**Officer Squad**: We know from earlier that their are NaN values present in this column.

**Arrest Flag**: Clean.  This is our Target because it denotes all Physical Arrests.

**Frisk Flag**: 478 place holders. Delete to improve interpretability. 

**Precinct/Sector/Beat**: Besides the '-' placeholder, there are also place holders for 99 and OOJ.  Also, 15 FK_ERRORS. 

For my sanity and ease of use, we'll quickly change the column names to be more "python friendly".

In [7]:
df.columns = ['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_yob',
       'officer_gender', 'officer_race', 'subject_perceived_race',
       'subject_perceived_gender', 'reported_date', 'reported_time',
       'initial_call_type', 'final_call_type', 'call_type', 'officer_squad',
       'arrest_flag', 'frisk_flag', 'precinct', 'sector', 'beat']
df.columns

Index(['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_yob',
       'officer_gender', 'officer_race', 'subject_perceived_race',
       'subject_perceived_gender', 'reported_date', 'reported_time',
       'initial_call_type', 'final_call_type', 'call_type', 'officer_squad',
       'arrest_flag', 'frisk_flag', 'precinct', 'sector', 'beat'],
      dtype='object')

# Scrubbing

Alright, let's get to scrubbing data! We'll start with the easy stuff and move up to the more difficult decisions.

## Dashes
Until we get into the modeling stage, we'll replace all '-' values with the string 'Unknown'

In [8]:
df = df.replace('-', 'Unknown')

In [9]:
df.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,officer_squad,arrest_flag,frisk_flag,precinct,sector,beat
0,Unknown,-1,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,Unknown,Unknown,Unknown,SOUTH PCT 1ST W - ROBERT,N,N,South,O,O2
1,Unknown,-1,20150000001463,28806,Field Contact,None,5670,1965,M,White,...,07:59:00,Unknown,Unknown,Unknown,NaN,N,N,Unknown,Unknown,Unknown
2,Unknown,-1,20150000001516,29599,Field Contact,None,4844,1961,M,White,...,19:12:00,Unknown,Unknown,Unknown,NaN,N,Unknown,Unknown,Unknown,Unknown
3,Unknown,-1,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,Unknown,Unknown,Unknown,NaN,N,N,Unknown,Unknown,Unknown
4,Unknown,-1,20150000001739,33155,Field Contact,None,6973,1977,M,White,...,00:41:00,Unknown,Unknown,Unknown,NaN,N,N,Unknown,Unknown,Unknown


## Officer Gender

In [10]:
df.officer_gender.value_counts()

M    38584
F     4917
N       11
Name: officer_gender, dtype: int64

As we've already established, we don't know if 'N' stands for 'Not Available', 'Not Disclosed', or even 'Non-Gender Binary'. Since it's such a small amount of data, we'll just drop it.

In [11]:
df = df[df['officer_gender'] != 'N']

df.officer_gender.value_counts()

M    38584
F     4917
Name: officer_gender, dtype: int64

## Officer Squad
Addressing the NaN values

In [12]:
df.officer_squad.isna().value_counts()

False    42968
True       533
Name: officer_squad, dtype: int64

In the interest of preserving as much data as possible, it might be in our best interest to drop this column. What squad an officer belongs to may not matter as much as the demographics of the police officer. We'll keep a copy of the original data frame just in case.

In [13]:
adf = df.copy()
adf = adf.drop('officer_squad', axis=1)
adf.columns

Index(['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_yob',
       'officer_gender', 'officer_race', 'subject_perceived_race',
       'subject_perceived_gender', 'reported_date', 'reported_time',
       'initial_call_type', 'final_call_type', 'call_type', 'arrest_flag',
       'frisk_flag', 'precinct', 'sector', 'beat'],
      dtype='object')

## Subject ID
The identity might affect the model if people arrested were repeat offenders

In [14]:
# subsetting data so analyze the duplicate Subject ID's

ids = adf[adf['subject_id'] > 3]
ids.head()

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_date,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat
1147,Unknown,11621995853,20190000404915,11622044618,Arrest,Unknown,7638,1989,M,White,...,2019-10-30T00:00:00,22:51:57,ARSON - IP/JO,--PROPERTY DEST (DAMG),911,Y,N,West,D,D2
1169,Unknown,7726713382,20200000109117,12800165868,Offense Report,Unknown,7808,1986,M,White,...,2020-03-31T00:00:00,11:45:40,SHOPLIFT - THEFT,--THEFT - SHOPLIFT,911,N,N,Unknown,Unknown,Unknown
1170,Unknown,7727213211,20190000287593,9587320855,Field Contact,Unknown,8649,1991,M,Hispanic or Latino,...,2019-08-04T00:00:00,13:22:42,TRESPASS,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,N,N,North,J,J2
1171,Unknown,7727213211,20190000287593,9587334107,Field Contact,Unknown,8649,1991,M,Hispanic or Latino,...,2019-08-04T00:00:00,13:45:33,TRESPASS,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,911,N,N,North,J,J2
1172,Unknown,7727213211,20190000367834,10558772346,Field Contact,Unknown,8459,1990,M,Hispanic or Latino,...,2019-10-02T00:00:00,12:39:08,"WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)",--DISTURBANCE - OTHER,911,N,N,West,D,D2


In [15]:
ids.shape

(8830, 22)

In [16]:
ids = ids[ids['arrest_flag'] == 'Y']
ids.shape

(2282, 22)

In [17]:
ids = ids[ids['subject_id'] == 7727430767]

In [18]:
ids

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_date,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat
24142,26 - 35,7727430767,20190000305890,9730643588,Arrest,Unknown,7782,1986,M,White,...,2019-08-17T00:00:00,15:52:35,THEFT (DOES NOT INCLUDE SHOPLIFT OR SVCS),--THEFT - CAR PROWL,911,Y,N,East,G,G2
24143,26 - 35,7727430767,20190000369340,10569538287,Arrest,Unknown,8491,1974,M,White,...,2019-10-03T00:00:00,15:06:56,Unknown,Unknown,Unknown,Y,Y,West,M,M3
24144,26 - 35,7727430767,20190000405808,11636833403,Arrest,Unknown,8389,1987,M,White,...,2019-10-31T00:00:00,17:51:36,SUSPICIOUS STOP - OFFICER INITIATED ONVIEW,--WARRANT SERVICES - FELONY,ONVIEW,Y,Y,Unknown,Unknown,Unknown
24146,26 - 35,7727430767,20200000149247,13111480477,Arrest,Unknown,8394,1991,M,White,...,2020-05-06T00:00:00,00:41:30,TRESPASS,--PROWLER - TRESPASS,911,Y,Y,North,U,U2
24147,26 - 35,7727430767,20200000180687,13305975975,Arrest,Unknown,8556,1995,M,White,...,2020-06-03T00:00:00,18:42:08,"DISTURBANCE, MISCELLANEOUS/OTHER","--ASSAULTS, OTHER",911,Y,N,West,D,D1
24149,26 - 35,7727430767,20200000215252,13871417277,Arrest,Knife/Cutting/Stabbing Instrument,8586,1990,M,White,...,2020-07-17T00:00:00,22:03:04,ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS),--DV - DOMESTIC VIOL/ASLT (ARREST MANDATORY),911,Y,Y,South,R,R3


Because these incidents have separate Terry Stop IDs, Dates, and different reporting officers, these are separate incidents and shouldn't effect our data.

### Feature Engineering: Repeat Offenders
Let's create a column that specifies people that have been stopped multiple times.

In [19]:
import numpy as np

adf['repeat_offenders'] = np.nan

for i in range(len(adf['subject_id'])):
    if adf['subject_id'].iloc[i] > 1:
        adf['repeat_offenders'].iloc[i] = 1
    else:
        adf['repeat_offenders'].iloc[i] = 0

adf['repeat_offenders'].value_counts()  

0.0    34671
1.0     8830
Name: repeat_offenders, dtype: int64

Awesome! Now let's turn those -1's in 'Subject_ID' to 'unassigned'

In [20]:
adf['subject_id'] = adf.subject_id.replace(-1, 'unassigned')
adf.subject_id.value_counts()

unassigned     34671
7726859935        18
7727117712        12
7753260438        11
7727827572         8
               ...  
8335944118         1
13499591239        1
7727655354         1
7745743294         1
12618563586        1
Name: subject_id, Length: 7039, dtype: int64

## General Offense/Street Check Number
Looking into why there are repeated values 

In [21]:
stop_chk = adf[adf['go_sc_num'] > 1]
stop_chk['go_sc_num'].value_counts()

20160000378750    16
20150000190790    16
20180000134604    14
20190000441736    13
20170000132836    13
                  ..
20180000333849     1
20170000130073     1
20200000020505     1
20170000189693     1
20180000071981     1
Name: go_sc_num, Length: 33908, dtype: int64

In [22]:
stop_chk = stop_chk[stop_chk['go_sc_num'] == 20160000378750]
stop_chk

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat,repeat_offenders
6521,18 - 25,unassigned,20160000378750,208302,Offense Report,None,7492,1983,M,White,...,22:31:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
6522,18 - 25,unassigned,20160000378750,208311,Arrest,None,7492,1983,M,White,...,23:04:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
16465,26 - 35,unassigned,20160000378750,208300,Offense Report,None,7492,1983,M,White,...,22:22:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
16466,26 - 35,unassigned,20160000378750,208301,Offense Report,None,7492,1983,M,White,...,22:24:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
16467,26 - 35,unassigned,20160000378750,208303,Offense Report,None,7492,1983,M,White,...,22:35:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
16468,26 - 35,unassigned,20160000378750,208307,Offense Report,None,7492,1983,M,White,...,22:48:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
29312,36 - 45,unassigned,20160000378750,208299,Offense Report,None,7492,1983,M,White,...,22:18:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
29313,36 - 45,unassigned,20160000378750,208305,Offense Report,None,7492,1983,M,White,...,22:41:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
29314,36 - 45,unassigned,20160000378750,208308,Offense Report,None,7492,1983,M,White,...,22:51:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0
29315,36 - 45,unassigned,20160000378750,208310,Offense Report,None,7492,1983,M,White,...,22:59:00,Unknown,Unknown,Unknown,N,Y,North,N,N3,0.0


From the dates, the separate Terry Stop ID's, the different Stop Resolutions and it all roughly happening within the same hour, it appears that this was a **dispute** of some sort in which an officer **collected Offense Reports from 12 people** and issued out **tickets to 4 people** (because there was **no physical arrest** denoted by the column 'arrest_flag', these were **non-custodial** arrests/citations).  

Looking back at the Column Description document, the GO/SC Number is considered the **"parent report"** that contain **associated Terry Stops**. This confirms my observations.  

## Terry Stop ID
There were duplicate ID's in here, so let's find out why.

In [23]:
dup_ids = adf[adf['terry_stop_id'].duplicated(keep=False)]
dup_ids = dup_ids[['subject_age_group', 'subject_id', 'go_sc_num', 
                   'terry_stop_id', 'stop_resolution', 'weapon_type',
                   'officer_id', 'reported_date', 'reported_time',
                   'initial_call_type', 'final_call_type', 'arrest_flag',
                   'frisk_flag', ]]
dup_ids

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,reported_date,reported_time,initial_call_type,final_call_type,arrest_flag,frisk_flag
3229,1 - 17,11637443950,20190000465104,12105013403,Arrest,Knife/Cutting/Stabbing Instrument,8500,2019-12-17T00:00:00,00:13:50,ROBBERY - IP/JO (INCLUDES STRONG ARM),--ROBBERY - ARMED,Y,Y
3230,1 - 17,11637443950,20190000465104,12105013403,Arrest,Mace/Pepper Spray,8500,2019-12-17T00:00:00,00:13:50,ROBBERY - IP/JO (INCLUDES STRONG ARM),--ROBBERY - ARMED,Y,Y
11383,18 - 25,7764935851,20200000080922,12689034912,Field Contact,Blunt Object/Striking Implement,7667,2020-03-04T00:00:00,19:52:40,"WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)","--WEAPON, PERSON WITH - GUN",N,Y
11384,18 - 25,7764935851,20200000080922,12689034912,Field Contact,Knife/Cutting/Stabbing Instrument,7667,2020-03-04T00:00:00,19:52:40,"WEAPN-IP/JO-GUN,DEADLY WPN (NO THRT/ASLT/DIST)","--WEAPON, PERSON WITH - GUN",N,Y
11688,18 - 25,9458419522,20190000285750,9585545373,Field Contact,Firearm,8382,2019-08-03T00:00:00,22:50:59,ASLT - IP/JO - PERSON SHOT OR SHOT AT,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,N,Y
11689,18 - 25,9458419522,20190000285750,9585545373,Field Contact,Handgun,8382,2019-08-03T00:00:00,22:50:59,ASLT - IP/JO - PERSON SHOT OR SHOT AT,--SUSPICIOUS CIRCUM. - SUSPICIOUS PERSON,N,Y
11882,18 - 25,11896724492,20190000454262,12034618758,Arrest,Knife/Cutting/Stabbing Instrument,7782,2019-12-08T00:00:00,15:41:10,DIST - IP/JO - DV DIST - NO ASLT,"--WEAPON, PERSON WITH - GUN",Y,Y
11883,18 - 25,11896724492,20190000454262,12034618758,Arrest,Other Firearm,7782,2019-12-08T00:00:00,15:41:10,DIST - IP/JO - DV DIST - NO ASLT,"--WEAPON, PERSON WITH - GUN",Y,Y
12037,18 - 25,12781633141,20200000136635,13080077761,Arrest,Blunt Object/Striking Implement,8680,2020-04-24T00:00:00,22:12:19,"WEAPN - GUN,DEADLY WPN (NO THRTS/ASLT/DIST)",--NARCOTICS - OTHER,Y,Y
12038,18 - 25,12781633141,20200000136635,13080077761,Arrest,Knife/Cutting/Stabbing Instrument,8680,2020-04-24T00:00:00,22:12:19,"WEAPN - GUN,DEADLY WPN (NO THRTS/ASLT/DIST)",--NARCOTICS - OTHER,Y,Y


Let's take a closer look at ID # 13080077761, as it also has an identical Subject ID and parent report (GO/SC Num)

In [24]:
dup_ids = dup_ids[dup_ids['terry_stop_id'] == 13080077761]
dup_ids

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,reported_date,reported_time,initial_call_type,final_call_type,arrest_flag,frisk_flag
12037,18 - 25,12781633141,20200000136635,13080077761,Arrest,Blunt Object/Striking Implement,8680,2020-04-24T00:00:00,22:12:19,"WEAPN - GUN,DEADLY WPN (NO THRTS/ASLT/DIST)",--NARCOTICS - OTHER,Y,Y
12038,18 - 25,12781633141,20200000136635,13080077761,Arrest,Knife/Cutting/Stabbing Instrument,8680,2020-04-24T00:00:00,22:12:19,"WEAPN - GUN,DEADLY WPN (NO THRTS/ASLT/DIST)",--NARCOTICS - OTHER,Y,Y
12039,18 - 25,12781633141,20200000136635,13080077761,Arrest,Mace/Pepper Spray,8680,2020-04-24T00:00:00,22:12:19,"WEAPN - GUN,DEADLY WPN (NO THRTS/ASLT/DIST)",--NARCOTICS - OTHER,Y,Y


We can infer from this data that these represent a **single person** who was carrying **multiple weapon types** and the officer had to **create separate entries for each weapon**.

This could pose a problem, as it inflates the number of stops.  There are only 31 of these values, so we'll drop the duplicates and keep the first instance. This will also **clean up** any people that are **wrongfully classified as 'Repeat Offenders'**.

In [25]:
adf = adf.drop_duplicates('terry_stop_id', keep='first')
adf

,subject_age_group,subject_id,go_sc_num,terry_stop_id,stop_resolution,weapon_type,officer_id,officer_yob,officer_gender,officer_race,...,reported_time,initial_call_type,final_call_type,call_type,arrest_flag,frisk_flag,precinct,sector,beat,repeat_offenders
0,Unknown,unassigned,20140000120677,92317,Arrest,None,7500,1984,M,Black or African American,...,11:32:00,Unknown,Unknown,Unknown,N,N,South,O,O2,0.0
1,Unknown,unassigned,20150000001463,28806,Field Contact,None,5670,1965,M,White,...,07:59:00,Unknown,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,0.0
2,Unknown,unassigned,20150000001516,29599,Field Contact,None,4844,1961,M,White,...,19:12:00,Unknown,Unknown,Unknown,N,Unknown,Unknown,Unknown,Unknown,0.0
3,Unknown,unassigned,20150000001670,32260,Field Contact,None,7539,1963,M,White,...,04:55:00,Unknown,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,0.0
4,Unknown,unassigned,20150000001739,33155,Field Contact,None,6973,1977,M,White,...,00:41:00,Unknown,Unknown,Unknown,N,N,Unknown,Unknown,Unknown,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43507,56 and Above,14268071835,20200000223475,14273492771,Arrest,Firearm,8731,1994,M,Asian,...,20:37:41,FIGHT - VERBAL/ORAL (NO WEAPONS),"--ASSAULTS - HARASSMENT, THREATS",911,Y,Y,East,G,G3,1.0
43508,56 and Above,14702844421,20200000231783,14704373070,Field Contact,Knife/Cutting/Stabbing Instrument,8462,1983,M,Two or More Races,...,23:07:39,FIGHT - IP/JO - WITH WEAPONS,--DISTURBANCE - FIGHT,911,N,Y,West,K,K3,1.0
43509,56 and Above,14817343569,20200000235500,14817848837,Offense Report,Unknown,8504,1986,M,White,...,12:02:39,PROPERTY - DAMAGE,--CRISIS COMPLAINT - GENERAL,911,N,N,West,K,K3,1.0
43510,56 and Above,14833970270,20200000238530,14849110521,Arrest,Unknown,8689,1987,M,White,...,00:36:07,ASLT - IP/JO - WITH OR W/O WPNS (NO SHOOTINGS),"--ASSAULTS, OTHER",911,Y,N,East,C,C2,1.0


## Report Date
We want to remove the time stamp from date, create a column for the year and finally calculate the ages for the officers 

In [26]:
adf.reported_date.dtype

dtype('O')

In [27]:
adf.reported_date.head()

0    2015-10-16T00:00:00
1    2015-03-19T00:00:00
2    2015-03-21T00:00:00
3    2015-04-01T00:00:00
4    2015-04-03T00:00:00
Name: reported_date, dtype: object

Ok, So we want to get rid of the 'T00:00:00' at the end of each entry, then convert this to date time, extract the year and put it into a new column, and use the new column as a reference to calculate the ages of each officer at the time of the report.  

In [28]:
# removing the time tag since it's useless 
adf['reported_date'] = adf['reported_date'].str.replace(r'T00:00:00$','')
adf.reported_date.head()

0    2015-10-16
1    2015-03-19
2    2015-03-21
3    2015-04-01
4    2015-04-03
Name: reported_date, dtype: object

In [29]:
# converting to date time
import datetime as dt

adf['reported_date'] = pd.to_datetime(adf['reported_date'])
# creating column containing the incident year 
adf['incident_year'] = adf['reported_date'].dt.year
adf['incident_year'].head()

0    2015
1    2015
2    2015
3    2015
4    2015
Name: incident_year, dtype: int64

In [30]:
adf['incident_month'] = adf['reported_date'].dt.month
adf['incident_month'].head()

0    10
1     3
2     3
3     4
4     4
Name: incident_month, dtype: int64

Let's drop "reported_date" now.

In [31]:
adf = adf.drop('reported_date', axis=1)

### Feature Engineering: Officer Age

Beautiful! Now lets create a column that calculates the age of the officer.

In [32]:
adf['officer_age'] = adf['incident_year'] - adf['officer_yob']
adf['officer_age']

0        31
1        50
2        54
3        52
4        38
         ..
43507    26
43508    37
43509    34
43510    33
43511    34
Name: officer_age, Length: 43485, dtype: int64

Wonderful!  The more we know about the officer's who reported to the scene, the better off we'll be.

## Stop Resolution 
We know that any resolution that is **labeled** "arrest" but is **not flagged** as one in the appropriate column is called a **"non-custodial arrest"**, or an instance where a **citation** was issued. To make the distinction more clear, we'll relabel these as such.

In [33]:
# first we'll check the value counts
adf.stop_resolution.value_counts()

Field Contact               17239
Offense Report              14868
Arrest                      10482
Referred for Prosecution      728
Citation / Infraction         168
Name: stop_resolution, dtype: int64

There is already a value for Citations, but we'll change these, too, since there must have been a reason they created the distinction. 

First, let's just double check that there **weren't any physical arrests** made when **citations** were issued out.

In [34]:
cits = adf[['stop_resolution', 'arrest_flag']]
cits = cits[cits['stop_resolution'] == 'Citation / Infraction']
cits

,stop_resolution,arrest_flag
31,Citation / Infraction,N
109,Citation / Infraction,N
212,Citation / Infraction,N
344,Citation / Infraction,N
1221,Citation / Infraction,N
...,...,...
42792,Citation / Infraction,N
43030,Citation / Infraction,N
43031,Citation / Infraction,N
43080,Citation / Infraction,N


In [35]:
cits.arrest_flag.value_counts()

N    168
Name: arrest_flag, dtype: int64

Ok, there were **no physical arrests** made where **citations** were issued. We're clear to make the change.

In [36]:
for i in range(len(adf['stop_resolution'])):
    if adf['stop_resolution'].iloc[i] == 'Arrest' and adf['arrest_flag'].iloc[i] == 'N':
        adf['stop_resolution'].iloc[i] = adf['stop_resolution'].iloc[i].replace('Arrest', 
                                               'Non-Custodial Arrest')
    
    elif adf['stop_resolution'].iloc[i] == 'Citation / Infraction':
        adf['stop_resolution'].iloc[i] = adf['stop_resolution'].iloc[i].replace('Citation / Infraction', 
                                               'Non-Custodial Arrest')

In [37]:
adf['stop_resolution'].value_counts()

Field Contact               17239
Offense Report              14868
Non-Custodial Arrest         8377
Arrest                       2273
Referred for Prosecution      728
Name: stop_resolution, dtype: int64

Look at how much the number of true arrests dropped! Let's cross-reference this with the number of Arrest Flags

In [38]:
adf.arrest_flag.value_counts()

N    41210
Y     2275
Name: arrest_flag, dtype: int64

Nice! The numbers line up very closely. There are only **2 physical arrests** that are **not accounted** for which means that they're **associated** with the **other** 'Stop Resolutions'. I don't see a reason to chase them down.  It's such a tiny number and I don't want to re-write an officer's paperwork. Only to clarify.

## Weapon Type
There is a lot of redundancy in here and so we're gonna do some cleaning up to simplify

In [39]:
adf.weapon_type.value_counts()

None                                 32560
Unknown                               8448
Lethal Cutting Instrument             1482
Knife/Cutting/Stabbing Instrument      422
Handgun                                270
Firearm Other                          100
Blunt Object/Striking Implement         53
Club, Blackjack, Brass Knuckles         49
Firearm                                 25
Firearm (unk type)                      15
Mace/Pepper Spray                       14
Other Firearm                           12
Club                                     9
Rifle                                    7
None/Not Applicable                      6
Taser/Stun Gun                           4
Shotgun                                  3
Automatic Handgun                        2
Fire/Incendiary Device                   2
Blackjack                                1
Brass Knuckles                           1
Name: weapon_type, dtype: int64

Let's work our way down.

In [40]:
for i in range(len(adf['weapon_type'])):
    if adf['weapon_type'].iloc[i] == 'None/Not Applicable':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('None/Not Applicable',
                                                                       'None')
    elif adf['weapon_type'].iloc[i] == 'Lethal Cutting Instrument':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Lethal Cutting Instrument',
                                                                       'Knife/Cutting/Stabbing Instrument')
    elif adf['weapon_type'].iloc[i] == 'Club, Blackjack, Brass Knuckles':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Club, Blackjack, Brass Knuckles',
                                                                       'Blunt Object/Striking Implement')
    elif adf['weapon_type'].iloc[i] == 'Club':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Club',
                                                                       'Blunt Object/Striking Implement')
    elif adf['weapon_type'].iloc[i] == 'Blackjack':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Blackjack',
                                                                       'Blunt Object/Striking Implement')
    elif adf['weapon_type'].iloc[i] == 'Brass Knuckles':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Brass Knuckles',
                                                                       'Blunt Object/Striking Implement')
    elif adf['weapon_type'].iloc[i] == 'Firearm Other':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Firearm Other',
                                                                       'Firearm')
    elif adf['weapon_type'].iloc[i] == 'Firearm (unk type)':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Firearm (unk type)',
                                                                       'Firearm')
    elif adf['weapon_type'].iloc[i] == 'Other Firearm':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Other Firearm',
                                                                       'Firearm')
    elif adf['weapon_type'].iloc[i] == 'Rifle':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Rifle',
                                                                       'Firearm')
    elif adf['weapon_type'].iloc[i] == 'Shotgun':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Shotgun',
                                                                       'Firearm')
    elif adf['weapon_type'].iloc[i] == 'Automatic Handgun':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Automatic Handgun',
                                                                       'Firearm')
    elif adf['weapon_type'].iloc[i] == 'Handgun':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Handgun',
                                                                       'Firearm')
    elif adf['weapon_type'].iloc[i] == 'Taser/Stun Gun':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Taser/Stun Gun',
                                                                       'Other')
    elif adf['weapon_type'].iloc[i] == 'Mace/Pepper Spray':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Mace/Pepper Spray',
                                                                       'Other')
    elif adf['weapon_type'].iloc[i] == 'Fire/Incendiary Device':
        adf['weapon_type'].iloc[i] = adf['weapon_type'].iloc[i].replace('Fire/Incendiary Device',
                                                                       'Other')

In [41]:
adf.weapon_type.value_counts()

None                                 32566
Unknown                               8448
Knife/Cutting/Stabbing Instrument     1904
Firearm                                434
Blunt Object/Striking Implement        113
Other                                   20
Name: weapon_type, dtype: int64

That was arduous, but I couldn't think of a way to make a function that was general enough.  At least it's cleaned up!

## Officer YOB 
Now that we have the age of the officers, we can just drop this column

In [42]:
bdf = adf.drop('officer_yob', axis=1)
bdf.columns

Index(['subject_age_group', 'subject_id', 'go_sc_num', 'terry_stop_id',
       'stop_resolution', 'weapon_type', 'officer_id', 'officer_gender',
       'officer_race', 'subject_perceived_race', 'subject_perceived_gender',
       'reported_time', 'initial_call_type', 'final_call_type', 'call_type',
       'arrest_flag', 'frisk_flag', 'precinct', 'sector', 'beat',
       'repeat_offenders', 'incident_year', 'incident_month', 'officer_age'],
      dtype='object')

## Reported Time
We'll extract the hour of the day with this data

In [43]:
bdf['reported_time'] = pd.to_datetime(bdf['reported_time'])
bdf['reported_hour'] = bdf['reported_time'].dt.hour
bdf.reported_hour.head()

0    11
1     7
2    19
3     4
4     0
Name: reported_hour, dtype: int64

Awesome! The hour column is made and based on a 24-hour clock.

Let's drop 'reported_time'. 

In [44]:
cdf = bdf.drop('reported_time', axis=1)
cdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43485 entries, 0 to 43511
Data columns (total 24 columns):
subject_age_group           43485 non-null object
subject_id                  43485 non-null object
go_sc_num                   43485 non-null int64
terry_stop_id               43485 non-null int64
stop_resolution             43485 non-null object
weapon_type                 43485 non-null object
officer_id                  43485 non-null object
officer_gender              43485 non-null object
officer_race                43485 non-null object
subject_perceived_race      43485 non-null object
subject_perceived_gender    43485 non-null object
initial_call_type           43485 non-null object
final_call_type             43485 non-null object
call_type                   43485 non-null object
arrest_flag                 43485 non-null object
frisk_flag                  43485 non-null object
precinct                    43485 non-null object
sector                      43485 non-null 

## Call types
As mentioned above, there are 12,000+ missing values which represent instances where the Computer Assisted Dispatch system was not used.  Keeping these values will cause a lot of noise since no reason is reported for stopping someone.  It's best to just drop this data to get rid of noise and have a more specific dataset.

In [45]:
cdf['call_type'].value_counts()

911                              19169
Unknown                          12817
ONVIEW                            8201
TELEPHONE OTHER, NOT 911          3018
ALARM CALL (NOT POLICE ALARM)      274
TEXT MESSAGE                         3
PROACTIVE (OFFICER INITIATED)        2
SCHEDULED EVENT (RECURRING)          1
Name: call_type, dtype: int64

While we're at it, let's drop those values that have such tiny counts.

In [46]:
ddf = cdf[(cdf['call_type'] != 'Unknown') & 
          (cdf['call_type'] != 'TEXT MESSAGE') &
         (cdf['call_type'] != 'PROACTIVE (OFFICER INITIATED)') & 
          (cdf['call_type'] != 'SCHEDULED EVENT (RECURRING)')]
ddf.shape

(30662, 24)

In [47]:
ddf.call_type.value_counts()

911                              19169
ONVIEW                            8201
TELEPHONE OTHER, NOT 911          3018
ALARM CALL (NOT POLICE ALARM)      274
Name: call_type, dtype: int64

Hate to see it all go, but it will be better in the end.

## Frisk Flag
Let's get rid of the missing values in frisk_flag.

In [48]:
ddf.frisk_flag.value_counts()

N          22522
Y           7854
Unknown      286
Name: frisk_flag, dtype: int64

In [49]:
ddf = ddf[ddf['frisk_flag'] != "Unknown"]
ddf.frisk_flag.value_counts()

N    22522
Y     7854
Name: frisk_flag, dtype: int64

In [50]:
col_values(ddf)

subject_age_group


26 - 35         10157
36 - 45          6364
18 - 25          6173
46 - 55          3836
56 and Above     1504
1 - 17           1473
Unknown           869
Name: subject_age_group, dtype: int64
-------------------------------------------------------


subject_id


unassigned     21840
7726859935        18
7727117712        12
7753260438        11
7726318196         8
               ...  
12602157491        1
8335944118         1
13499591239        1
7727655354         1
12618563586        1
Name: subject_id, Length: 6843, dtype: int64
-------------------------------------------------------


go_sc_num


20150000190790    16
20180000134604    14
20170000132836    13
20190000441736    13
20200000025654    11
                  ..
20190000157419     1
20200000121158     1
20190000456118     1
20200000120273     1
20170000251761     1
Name: go_sc_num, Length: 23559, dtype: int64
-------------------------------------------------------


terry_stop_id


460799         1
9736